In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score

In [3]:
train_raw = pd.read_csv('train_pr.csv')
test = pd.read_csv('test_pr.csv')

In [4]:
split_date = '2014-03-01'
# разбиваем данные на обучающий и отложенный набор данных (train и out of time)
train = train_raw[train_raw['timestamp'] < split_date]
oot = train_raw[train_raw['timestamp'] >= split_date]

# создаем колонку по которой будем делать стратификацию данных
stratify = train['timestamp'].str[:-3] + '_' + train['__churn'].map(str)

# разбиваем датасет на обучающий и отложенный (train и out of sample) в соотношении 70% (train) на 30% (out of time) использую стратифицированную колонку
train, oos = train_test_split(train, test_size=0.3, random_state=47, stratify=stratify)

train.shape, oos.shape, oot.shape

((10906, 50), (4675, 50), (4902, 50))

In [5]:
remove_features = train.columns[train.columns.str.startswith('__')].tolist()
remove_features

['__churn', '__price_doc']

In [6]:
# формируем колонки с непрерывными признаками, которые будем подавать в модель
continuous_features = list(set(train.dtypes[train.dtypes != 'object'].index.tolist()) 
                           - set(remove_features))
# continuous_features

In [7]:
X_train = train[continuous_features].fillna(0.).copy(True)
X_test = test[continuous_features].fillna(0.).copy(True)
X_oos = oos[continuous_features].fillna(0.).copy(True)
X_oot = oot[continuous_features].fillna(0.).copy(True)

# список в котором храним набор различных датасетов для обучения и оценки модели
# (имя датасета, обучающий набор фичей, отложенный набор фичей)
# первым должен стоять обучающий набор данных
X_y_datasets = [
    ('train', X_train, train),
    ('oos', X_oos, oos),
    ('oot', X_oot, oot)]

In [12]:
name, X_, y_ = X_y_datasets[0]

df_boost_clf = pd.DataFrame({'number': [],'alpha': [], 'train': [], 'oos': [], 'oot': []})

log_model = GradientBoostingClassifier(random_state=47,
        max_depth=4,
        n_estimators=100)
log_model.fit(X_, y_['__churn'])

for name, X_, y_ in X_y_datasets:
    # получаем предсказания модели
    value = []
    for j in range(len(log_model.predict_proba(X_))):
        value.append(log_model.predict_proba(X_)[j][1])
        #value[value < 0] = 0
    y_['__churn_predict_log'] = value
    
    
new=[100,4]
    
    
for name, X_, y_ in X_y_datasets:
    # получаем оценку качества модели
    value = roc_auc_score(y_['__churn'], y_['__churn_predict_log'])
        #print(f'{name:10s}: {value:.4f}')
    new.append(value)
df_boost_clf.loc[ len(df_boost_clf.index )] = new
    
    
df_boost_clf["mean"]=(df_boost_clf['oos']+df_boost_clf['oot'])/2
df_boost_clf

C:\Users\1\AppData\Local\Temp\ipykernel_22488\3515887215.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_['__churn_predict_log'] = value


,number,alpha,train,oos,oot,mean
0,100.0,4.0,0.984977,0.972158,0.966728,0.969443


In [15]:
name, X_, y_ = X_y_datasets[0]

df_boost = pd.DataFrame({'number': [],'alpha': [], 'train': [], 'oos': [], 'oot': []})

reg_model = GradientBoostingRegressor(random_state=47,
        max_depth=9,
        n_estimators=99)
reg_model.fit(X_, y_['__price_doc'])

for name, X_, y_ in X_y_datasets:
    # получаем предсказания модели
    value = reg_model.predict(X_)
    # зануляем значения прогноза в случае если они ниже 0
    value[value < 0] = 0
    y_['__price_predict_decisiontree'] = value
    
    
new=[99,9]
    
    
for name, X_, y_ in X_y_datasets:
    # получаем оценку качества модели
    value = mean_squared_log_error(y_true=y_['__price_doc'], y_pred=y_['__price_predict_decisiontree'])
        #print(f'{name:10s}: {value:.4f}')
    new.append(value)
df_boost.loc[ len(df_boost.index )] = new
    
    
df_boost["mean"]=(df_boost['oos']+df_boost['oot'])/2
df_boost

C:\Users\1\AppData\Local\Temp\ipykernel_22488\1110163082.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_['__price_predict_decisiontree'] = value


,number,alpha,train,oos,oot,mean
0,99.0,9.0,0.036218,0.139689,0.144507,0.142098


In [20]:
name, X_, y_ = X_y_datasets[0]

X_ = X_.copy(True)
# Создаем новую колонку в датасете (для примера с именем "random")
# Колонка никак не связана с целевой переменной
# и имеет равномерное распределение от 0 до 1
X_["random"] = np.random.normal()
#я использую стандартное нормальное распределение, потому что признаки стандартизованные и часто могут быть нормальны
#практика показала, что остается меньше признаков
train_columns = X_.columns.tolist()

# создаем объект модель
regr_alt = GradientBoostingRegressor(random_state=47,
        max_depth=9,
        n_estimators=99)

# обучаем модель на всех признаках
regr_alt.fit(X_, y_['__price_doc'])

# получаем важность признаков
df = pd.DataFrame({"feature": X_.columns, "importance": regr_alt.feature_importances_})
# получаем важность признака random
rv_importance = df[df["feature"] == "random"]["importance"].iloc[0]

# оставляем признаки с важностью равной или большей чем случайный признак
sel_cols_reg = df[df["importance"] > rv_importance]["feature"].tolist()
# выводим кол-во признаков c важностью больше чем случайная
print(f"Number of features: {len(sel_cols_reg)}")
# выводим имена признаков
print(f"Names of features: {sel_cols_reg}")

Number of features: 47
Names of features: ['total_revolving_bal', 'public_transport_station_min_walk', 'avg_open_to_buy', 'railroad_1line', 'railroad_terminal_raion', 'mkad_km', 'avg_utilization_ratio', 'detention_facility_km', 'months_inactive_12_mon', 'months_on_book', 'marital_status', 'floor', 'customer_age', 'thermal_power_plant_raion', 'total_amt_chng_q4_q1', 'office_sqm_5000', 'leisure_count_5000', 'nuclear_reactor_raion', 'radiation_raion', 'railroad_station_walk_km', 'water_1line', 'school_education_centers_top_20_raion', 'product_type', 'market_count_1500', 'big_market_raion', 'oil_chemistry_raion', 'total_ct_chng_q4_q1', 'full_sq', 'total_relationship_count', 'leisure_count_3000', 'gender', 'education_level', 'incineration_raion', 'total_trans_ct', 'trc_count_2000', 'card_category', 'big_road1_1line', 'culture_objects_top_25', 'credit_limit', 'income_category', 'detention_facility_raion', 'sport_count_2000', 'ecology', 'contacts_count_12_mon', 'metro_km_walk', 'dependent_cou

In [21]:
name, X_, y_ = X_y_datasets[0]

X_ = X_.copy(True)
# Создаем новую колонку в датасете (для примера с именем "random")
# Колонка никак не связана с целевой переменной
# и имеет равномерное распределение от 0 до 1
X_["random"] = np.random.normal()
#я использую стандартное нормальное распределение, потому что признаки стандартизованные и часто могут быть нормальны
#практика показала, что остается меньше признаков
train_columns = X_.columns.tolist()

# создаем объект модель
regr_alt = GradientBoostingClassifier(random_state=47,
        max_depth=4,
        n_estimators=100)

# обучаем модель на всех признаках
regr_alt.fit(X_, y_['__churn'])

# получаем важность признаков
df = pd.DataFrame({"feature": X_.columns, "importance": regr_alt.feature_importances_})
# получаем важность признака random
rv_importance = df[df["feature"] == "random"]["importance"].iloc[0]

# оставляем признаки с важностью равной или большей чем случайный признак
sel_cols_clf = df[df["importance"] > rv_importance]["feature"].tolist()
# выводим кол-во признаков c важностью больше чем случайная
print(f"Number of features: {len(sel_cols_clf)}")
# выводим имена признаков
print(f"Names of features: {sel_cols_clf}")

Number of features: 41
Names of features: ['total_revolving_bal', 'public_transport_station_min_walk', 'avg_open_to_buy', 'railroad_1line', 'railroad_terminal_raion', 'mkad_km', 'avg_utilization_ratio', 'detention_facility_km', 'months_inactive_12_mon', 'months_on_book', 'marital_status', 'floor', 'customer_age', 'thermal_power_plant_raion', 'total_amt_chng_q4_q1', 'office_sqm_5000', 'leisure_count_5000', 'nuclear_reactor_raion', 'radiation_raion', 'railroad_station_walk_km', 'water_1line', 'school_education_centers_top_20_raion', 'market_count_1500', 'big_market_raion', 'total_ct_chng_q4_q1', 'full_sq', 'total_relationship_count', 'leisure_count_3000', 'gender', 'total_trans_ct', 'trc_count_2000', 'card_category', 'culture_objects_top_25', 'credit_limit', 'income_category', 'sport_count_2000', 'ecology', 'contacts_count_12_mon', 'metro_km_walk', 'dependent_count', 'cafe_count_1000_price_1000']


In [ ]:
#проверим точность только с важными признаками

In [22]:
name, X_, y_ = X_y_datasets[0]

df_boost_clf_restr = pd.DataFrame({'number': [],'alpha': [], 'train': [], 'oos': [], 'oot': []})

log_model = GradientBoostingClassifier(random_state=47,
        max_depth=4,
        n_estimators=100)
log_model.fit(X_[sel_cols_clf], y_['__churn'])

for name, X_, y_ in X_y_datasets:
    # получаем предсказания модели
    value = []
    for j in range(len(log_model.predict_proba(X_[sel_cols_clf]))):
        value.append(log_model.predict_proba(X_[sel_cols_clf])[j][1])
        #value[value < 0] = 0
    y_['__churn_predict_log'] = value
    
    
new=[100,4]
    
    
for name, X_, y_ in X_y_datasets:
    # получаем оценку качества модели
    value = roc_auc_score(y_['__churn'], y_['__churn_predict_log'])
        #print(f'{name:10s}: {value:.4f}')
    new.append(value)
df_boost_clf_restr.loc[ len(df_boost_clf_restr.index )] = new
    
    
df_boost_clf_restr["mean"]=(df_boost_clf_restr['oos']+df_boost_clf_restr['oot'])/2
df_boost_clf_restr

C:\Users\1\AppData\Local\Temp\ipykernel_22488\2924774811.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_['__churn_predict_log'] = value


,number,alpha,train,oos,oot,mean
0,100.0,4.0,0.984977,0.972152,0.966724,0.969438


In [23]:
name, X_, y_ = X_y_datasets[0]

df_boost_restr = pd.DataFrame({'number': [],'alpha': [], 'train': [], 'oos': [], 'oot': []})

reg_model = GradientBoostingRegressor(random_state=47,
        max_depth=9,
        n_estimators=99)
reg_model.fit(X_[sel_cols_reg], y_['__price_doc'])

for name, X_, y_ in X_y_datasets:
    # получаем предсказания модели
    value = reg_model.predict(X_[sel_cols_reg])
    # зануляем значения прогноза в случае если они ниже 0
    value[value < 0] = 0
    y_['__price_predict_decisiontree'] = value
    
    
new=[99,9]
    
    
for name, X_, y_ in X_y_datasets:
    # получаем оценку качества модели
    value = mean_squared_log_error(y_true=y_['__price_doc'], y_pred=y_['__price_predict_decisiontree'])
        #print(f'{name:10s}: {value:.4f}')
    new.append(value)
df_boost_restr.loc[ len(df_boost_restr.index )] = new
    
    
df_boost_restr["mean"]=(df_boost_restr['oos']+df_boost_restr['oot'])/2
df_boost_restr

C:\Users\1\AppData\Local\Temp\ipykernel_22488\2427419363.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_['__price_predict_decisiontree'] = value


,number,alpha,train,oos,oot,mean
0,99.0,9.0,0.036218,0.139689,0.144507,0.142098


In [ ]:
#ну вот, в общем то потерь в качестве нет, а исключили из обеих моделей примерно 40 признаков